# Never a sity so real


In [1]:
import env
import utils

# We use Gemini directly instead through langchain
# because of safety settings,
# langchain is developing this and not hasn't released yet
import google.generativeai as genai
import os

genai.configure(
    api_key=os.environ.get("GOOGLE_API_KEY")
)
llm_gemini = genai.GenerativeModel(
    model_name = "gemini-pro",
    safety_settings=utils.safety_config
)


In [14]:
file_name = "Never a City So Real A Walk in Chicago (Alex Kotlowitz) (Z-Library)"
input_file = f"Chicago/{file_name}.pdf"
output_file = f"Chicago/{file_name}-cn.txt"

In [15]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(input_file)
pages = loader.load_and_split()

In [18]:
file_output = open(output_file, "a")

for index, page in enumerate(pages[:]):
    prompt = f""" Suppose you are a translator, and your target language is simplified Chinese, 
    Your audiences are Chinese native young students in Junior school about 13 years old. 
    Translate the following english text to simplified Chinese: \ ```{page.page_content}``` """ 
    
    response = llm_gemini.generate_content(
        prompt,
        safety_settings=utils.safety_config
    )

    if response == None:
        break
    file_output.write(f"\n---- page: {index} ---- \n")
    for p in response.parts:
        file_output.write(p.text.strip())
        file_output.flush()

file_output.close()

ValueError: The `response.parts` quick accessor only works for a single candidate, but none were returned. Check the `response.prompt_feedback` to see if the prompt was blocked.

In [24]:
for index, page in enumerate(pages[119:123]):
    prompt = f""" Suppose you are a translator, and your target language is simplified Chinese, 
    Your audiences are Chinese native young students in Junior school about 13 years old. 
    Translate the following english text to simplified Chinese: \ ```{page.page_content}``` """ 
    
    response = llm_gemini.generate_content(
        prompt,
        safety_settings=utils.safety_config
    )

    if len(response.parts) == 0:
        continue
    for p in response.parts:
        print(p.text.strip())

（汤普森最喜欢的夜场之一便是位于塞塞罗的拉尔夫·卡彭的棉花俱乐部。）卡彭将他的行动基地迁回芝加哥，但在塞塞罗仍然有很重要的势力，直到他在 1932 年入狱为止。这是一种令人恼火的联系，尤其是因为它总是媒体对这个地方的第一印象。1953 年，塞塞罗考虑更改其名称，因为，该镇的律师说，“各地的人都认为我们只是一群暴徒。塞塞罗的孩子无法进入大学兄弟会。”直到 1993 年，仍然希望让记者不再关注卡彭的联系，该镇悬挂横幅声称塞塞罗是欧内斯特·海明威的出生地。实际上，海明威来自附近的橡树公园，该公园在 1899 年未合并，因此在技术上（如果不是社会上）是塞塞罗的一部分。对卡彭的防御是可以理解的，但它似乎是错位的，因为塞塞罗声名狼藉的另一个原因在很大程度上更令人尴尬。简单地说，几十年来，塞塞罗一直奉行不允许黑人的政策。它像美国南部民权运动之前的任何事情一样激烈和有力。我敢打赌，这个小镇是在不让非裔美国人进入方面全国最擅长的。多年来，该村拒绝接受某些联邦补助金，因此它不必遵守公开住房或招聘法律。在那里长大的立陶宛人和意大利人告诉我，当他们还是孩子的时候，他们会将见到的任何黑人赶出城——在警察的授意下。1951年，伊利诺斯的亚历克斯·科特洛维茨
当三天的暴徒向刚刚搬进来的黑人公共汽车司机和他的家人的二层公寓投掷石块和碎金属时，州长阿德莱·史蒂文森不得不出动国民警卫队。城镇警察袖手旁观，据说有些还提供了石块，这些西塞罗人最终冲进公寓，将这个家庭的物品，包括钢琴从窗外扔了出去。1967年，马丁路德金在芝加哥领导其开放住房竞选活动，宣布将在西塞罗游行，以此让抵制金博士努力的芝加哥市长戴利难堪。但是当县里官员告诉金无法保障他的安全后，他选择不游行，在此之前，他曾面对伯明翰的警察局长布尔·康纳。金将西塞罗称为北方的塞尔玛。然后，在 1980 年代，一名警长和搬到西塞罗的少数几个非裔美国家庭之一发生冲突，司法部下令警方接受种族敏感性培训。我岳父杰克帮助领导了这些研讨会，他回忆起一名警长身穿印有“police brutality. The fun part of police work”（警察暴行。警察工作的有趣之处）字样的T恤参加会议。这是博伊尔搬到这里时所面临的情况，但一开始他和西塞罗看上去势均力敌。在越南担任过海军陆战队员的博伊尔体型像个橄榄球运动员，矮壮且双肩宽厚。他有着斗牛犬一样红润的

ValueError: The `response.parts` quick accessor only works for a single candidate, but none were returned. Check the `response.prompt_feedback` to see if the prompt was blocked.